## Medical History of British India: Mental Health

This notebook explores the Medical History of British India Mental Health.

This explorations aims to ask the data the following questions:
- when did this occur?
- what is an asylum?
- where were these asylums located in British India?
- how was a lunatic defined? what constituted mental health issues during the 19th century?
- who was put in lunatic asylums?
- what was the colonial context of psychiatry in British India and colonial ideas about the native body?


In python, using the NLTK library we use the concordance function to see words used in context

https://blog.ouseful.info/2015/12/13/n-gram-phrase-based-concordances-in-nltk/


In [1]:
# to access files for cleaning
import os, ssl 
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context

# Libraries for data loading
import pandas as pd  # For dataframe analysis
import numpy as np  # For dataframe analysis
import string
import re  # Regix to remove punctuation from strings I split

# Libraries for visualization
import altair as alt
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Libraries for text analysis
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')
from nltk.corpus import PlaintextCorpusReader
nltk.download('wordnet')
from nltk.corpus import wordnet
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.text import Text
from nltk.stem.porter import PorterStemmer
from nltk.probability import FreqDist
nltk.download('averaged_perceptron_tagger')
from nltk.tag import pos_tag
nltk.download('tagsets')  # part of speech tags
from nltk.draw.dispersion import dispersion_plot as displt


#Libraries for advanced text processing
from nltk.util import ngrams
from collections import Counter
nltk.download('averaged_perceptron_tagger')
from shutil import copyfile # For copying clean files
from sklearn.feature_extraction.text import CountVectorizer # For creating document-term matrix & excluding stop words
from sklearn.feature_extraction import text # For getting stop words
from wordcloud import WordCloud # For creating word clouds
from textblob import TextBlob # For sentiment analysis

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tamaralottering/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tamaralottering/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tamaralottering/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/tamaralottering/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to
[nltk_data]     /Users/tamaralottering/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/tamaralottering/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date

In [2]:
!pip install altair

In [ ]:
#create a corpus from all texts and tokenize
corpus_folder = '/Users/tamaralottering/Desktop/nls-text-indiaPapers/'
wordlists = PlaintextCorpusReader(corpus_folder, '\d.*', encoding='latin1')
corpus_tokens = wordlists.words()

#### Filter corpus for mental health texts

In [9]:
# Modify this list to add keywords 🧚
keywords = ['lunatic', 'asylum'] 

In [4]:
# walk through the /data folder and read text files to make a df
textList = []
for dirname, _, filenames in os.walk('/Users/tamaralottering/Desktop/nls-text-indiaPapers'):
    for filename in filenames:
        # print(os.path.join(dirname, filename))
        myfile = os.path.join(dirname, filename)
        with open(myfile, 'rb') as fopen:
            q = fopen.read().decode('ISO-8859-1')
            textList.append(q)
uncleanDf = pd.DataFrame(textList)
uncleanDf.columns = ['text']
uncleanDf

,text
0,IP/QB.10 m.91.b. No. 44. (NEW SERIES.) SCIENTI...
1,IP/6/HG.s4. REPORT ON THE CALCUTTA MEDICAL INS...
2,"CHOLERA IN INDIA, 1862 TO 1881. BENGAL PROVINC..."
3,Vol. I 1931 THE Indian Journal of Veterinary S...
4,"IP/QB, 10 m.91.b No. 19. (NEW SERIES.) SCIENTI..."
...,...
465,[NLS note: a graphic appears here - see image ...
466,REPORT ON THE WORKING OF THE MENTAL HOSPITALS ...
467,ICAR. 15. VIII. 650 Vol. VIII 1938 THE Indian ...
468,SLEEPING SICKNESS A SUMMARY OF THE WORK DONE B...


In [5]:
# Initial cleaning
def initialCleaning(mystring):
    mystring = mystring.lower() # Text normalization: make string lowercase
    mystring = re.sub(r'[^\w\s]','', mystring) # Text normalization: remove punctuation
    mystring = re.sub('\[.*?\]', '', mystring) #Text normalization: remove text in square brackets
    mystring = re.sub('https?://\S+|www\.\S+', '', mystring) #Text normalization: remove links
    mystring = re.sub('\n', '', mystring) #Text normalization: remove linebreaks
    mystring = re.sub('\w*\d\w*', '', mystring)#Text normalization: 
    return mystring

def countWords(string, wordsToCount):
    splitString = string.split() # Split string into array of words
    counts = Counter(splitString) # Get counts for each word like Counter({'dogs': 3, 'cute': 1})
    count = 0 # Start the counter
    for word in wordsToCount: # Loop through list of words and add the count
        count = count + counts[word]
    return count

In [6]:
cleanText = lambda text: initialCleaning(text) # Lambda function applies to all cells in a column

In [7]:
cleanDf = pd.DataFrame(uncleanDf.text.apply(cleanText)) # .apply() the function to all cells

In [10]:
# create a list of lunatic asylum texts
cleanerDfList = []
for index, row in cleanDf.iterrows():
    count = countWords(cleanDf['text'].iloc[index], keywords)
    if(count>4):
        cleanerDfList.append(cleanDf['text'].iloc[index])

In [11]:
# convert text to a dataframe to use where needed
cleanerDf = pd.DataFrame(cleanerDfList)
cleanerDf.columns = ['text']
cleanerDf

,text
0,gtx \rsi the phi price\rstatistical returns o...
1,triennial report of the lunatic asylums under ...
2,report on the lunatic asylums under the govern...
3,indian hemp drugs commission vol vi evidence o...
4,leprosy and its control in the bombay presiden...
...,...
61,no a proceedings of the honble the lieuten...
62,report on the working of the micro biological ...
63,annual administration and progress report on t...
64,annual administration and progress report on t...


#### Phrase Concordance in Mental Health Texts

In [13]:
# reconfigured solution from https://stackoverflow.com/questions/33813405/concordance-for-a-phrase-using-nltk-in-python
def n_concordance_tokenised(text,phrase,left_margin=5,right_margin=5):
    
    phraseList=phrase.split(' ')

    c = nltk.ConcordanceIndex(text.tokens, key = lambda s: s.lower())
    
    offsets=[c.offsets(x) for x in phraseList]
    offsets_norm=[]
    
    for i in range(len(phraseList)):
        offsets_norm.append([x-i for x in offsets[i]])
        
    intersects=set(offsets_norm[0]).intersection(*offsets_norm[1:])
    
    concordance_txt = ([text.tokens[list(map(lambda x: x-left_margin if (x-left_margin) > 0 else 0,[offset]))[0]:offset+len(phraseList)+right_margin]
                    for offset in intersects])

    outputs=[''.join([x+' ' for x in con_sub]) for con_sub in concordance_txt]
    return outputs

def n_concordance(txt,phrase,left_margin=5,right_margin=5):
    tokens = nltk.word_tokenize(txt)
    text = nltk.Text(tokens)
    return

In [14]:
raw_text = " ".join(cleanerDfList)

In [16]:
tokens = nltk.word_tokenize(raw_text)
text = nltk.Text(tokens)

### Where were lunatic asylums in British India?

In [17]:
n_concordance_tokenised(text,'Lunatic Asylum')

['in the buildings of the lunatic asylum at ahmedabad the improvements to ',
 'sanctioned establishment of the patna lunatic asylum as it stood on the ',
 'patients admitted into the patna lunatic asylum at bankipore during the year ',
 'j fairweather md surgeonmajor superintendent lunatic asylum no statement showing the sanctioned ',
 'patients admitted into the delhi lunatic asylum during the year total dlstrict ',
 'to the premises of the lunatic asylum which it adjoins it is ',
 'patients admitted into the punjab lunatic asylum lahore during the year and ',
 'the construction of a conjoined lunatic asylum is under consideration it is ',
 'patients admitted into the punjab lunatic asylum lahore during the tear and ',
 'scriven surgeonmajor the january superintendent lunatic asylum no return showing the types ',
 'sanctioned establishment of the lahore lunatic asylum as it stood on the ',
 'b boyd superintend nt colaba lunatic asylum bombay about seventeen years service ',
 'of medic

Lunatic Asylums were later called Mental Hospitals in the 19th century:

- mental hospital dharwar 
- mental hospital at tezpur
- ranchi indian mental hospital kanke in bihar and orissa
- patna mental hospital
- mental hospital ahmedabad
- punjab mental hospital lahore 
- central mental hospital yeravda
- madras mental hospital 
- ratnagiri mental hospital
- dharwar mental hospital 
- hyderabad mental hospital
- nagpur mental hospital
- vizagapatam mental hospital
- narotamdas madhavdas mental hospital naupáda
- barhampur central mental hospital 
- dacca mental hospital 
- berhampur cetral mental hospital
- waltair mental hospital

In [18]:
n_concordance_tokenised(text,'Mental Hospital')

['hospital hubli p ophthal mic mental hospital dharwar g mental broach and ',
 'males females total ranchi indian mental hospital ditto ditto b o g ',
 'the working of the patna mental hospital for the year statement i ',
 'account of manufactures in the mental hospital at tezpur ih year mean ',
 'patients in the ranchi indian mental hospital kanke during the year and ',
 'the patients admitted in the mental hospital j at texpur during the ',
 'noiv admitted into the punjab mental hospital and two previous years christians ',
 'su o t jfftmcopy of mental hospital d m a quot rt ',
 'p rs a p indian mental hospital kanke ranchi name of asylum ',
 'with the working of the mental hospital in rangoon on the st ',
 'occupied portion of the new mental hospital at tadagale rangoon and the ',
 'staff and inmates of the mental hospital but the improve ment of ',
 'working of the ranchi indian mental hospital kanke in bihar and orissa ',
 'tadagale rangoon and the temporary mental hospital at min

In [45]:
n_concordance_tokenised(text,'european mental hospital')

['and were as follows ftanchi european mental hospital ranobi indian mental hospital central ',
 'and management of the ranchi european mental hospital to the board of trustees ',
 'central provinces idhgill assam janehi european mental hospital eancbi indian mental hospital i ',
 'the report of the ranchi european mental hospital should be submitted to them ',
 'since the opening of the european mental hospital at ranchi in the year ',
 'this statement transferred to ranchi european mental hospital i g c h statement ',
 'fluriag the past years jaqchi european mental hospital amjohi indian mental hospital i ']

Looks like there were the following asylums:
- agra lunatic asylum 
- Benares Lunatic Asylum'
- cuttack lunatic asylum
- tezpur lunatic asylum
- Madras Lunatic Asylum 
- Colaba Lunatic Asylum
- Waltair Lunatic Asylum
- dacca lunatic asylum
- berhampore lunatic asylum 
- nagpur lunatic asylum 
- dhárwár lunatic asylum
- rangoon lunatic asylum
- delhi lunatic asylum
- lunatic asylum at ahmedabad
- ahmedabad lunatic asylum bombay
- patna lunatic asylum at bankipore
- patna lunatic asylum in bihar and orissa
- punjab lunatic asylum lahore 
- tiie lunatic asylum in assam
- ranchi european lunatic asylum
- lahore lunatic asylum 
- vizagapatam lunatic asylum 
- moydapore lunatic asylum
- jubbulpore lunatic asylum
- bhawanipur/bhowanipore lunatic asylum
- central lunatic asylum yeravda dharwar ahmedabad hyderabad
- central lunatic asylum yerávda ahmednagar ajmer allahabad aurangabad
- bareilly lunatic asylum benares
- lunatic asylum haidarabad
- lunatic asylum poona
- lunatic asylum jubbulpore
- central lunatic asylum for natives at ranchi
- calicut lunatic asylum

### Who were the occupants of lunatic asylums?

There also seemed to be a distinction between lunatic asylums for 'Europeans' and 'Natives'. The following were specified as Europeans' and 'Natives' asylums explicitly in the text, however there may have been more in each categoy or perhaps mixed asylums:

European Lunatic Asylums:
- european lunatic asylum bhawanipur
- ranchi european lunatic asylum  
- european lunatic asylum at colaba 

Native Lunatic Asylums:
- central lunatic asylum for natives at ranchi
- native lunatic asylum calcutta 
-  dullunda native lunatic asylum

In [21]:
n_concordance_tokenised(text,'lunatic asylum for natives')

['the construction of a central lunatic asylum for natives at ranchi which was submitted ']

In [23]:
n_concordance_tokenised(text,'native lunatic asylum')

['appendix diet roll of the native lunatic asylum at for date no of ',
 'contained the superintendent of the native lunatic asylum calcutta has suggested that the ',
 'hospitals bengal on the dullunda native lunatic asylum on april i inspected the ']

In [22]:
n_concordance_tokenised(text,'european lunatic asylum')

['people committed to their charge european lunatic asylum bhawanipur the following table gives ',
 'superintendent the management of the european lunatic asylum at bhowanipore is satisfactory during ',
 'lunatic asylums viz the ranchi european lunatic asylum and the patna lunatic asylum ',
 'in october intimated that the european lunatic asylum at ranchi is intended for ',
 'afflicted with a dreadful disease european lunatic asylum bhawanipur the following table gives ',
 'females were transferred to the european lunatic asylum at ranchi h inclusive of ',
 'two patients in the ranchi european lunatic asylum no x province of bihar ',
 'period under review the ranchi european lunatic asylum the ranchi european lunatic asylum ',
 'european lunatic asylum the ranchi european lunatic asylum was opened on the may ',
 'of the staff at ranchi european lunatic asylum are reported to be insufficient ',
 'for admission into the ranchi european lunatic asylum they ruled that europeans and ',
 'of

#### Criminal Lunatics
The British India Paper texts also mention 'criminal lunatics'. It is unclear at this point how they are different from other 'lunatics'.
- they may have been admitted to the lunatic asylums based on crimes or 'mischief'
- There may have been separate wards for 'criminal lunatics'
- these patients are described to have very aggressive demeanors in the text - they've killed, injured , escaped, a number committed suicide.

In [28]:
n_concordance_tokenised(text,'criminal lunatic')

['mortality reptiles at patna a criminal lunatic who had been under treatment ',
 'betul district this was a criminal lunatic a female the order directing ',
 'of one heera poonja a criminal lunatic who has now been quite ',
 'compared with in and in criminal lunatic harka committed suicide by hanging ',
 'the year no ma than criminal lunatic escaped during the night of ',
 'two were killed by a criminal lunatic with the iron head piece ',
 'for the year iireturn of criminal lunatic in the m on til ',
 'the preceding year c escapesone criminal lunatic named lai das fakir received ',
 'of government treated as a criminal lunatic he has since died from ',
 'ganja mania this is a criminal lunatic the case book shows that ',
 'age criminal lunatics only one criminal lunatic is in confinement and i ',
 'readmitted the first was a criminal lunatic named dina twice received from ',
 'this respect when the proposed criminal lunatic ward has been built the ',
 'how it was that the criminal luna

In [29]:
n_concordance_tokenised(text,'coolie criminal lunatic')

['in the large intestine suicidebudhan coolie criminal lunatic crime culpable homicide section indian ',
 'of ancemia post mortem phthisissajan coolie criminal lunatic received from dibrugarh admitted into ']

#### Coolie

The word 'Coolie' appears in the text a lot.

The word coolie was first popularized in the 16th century by European traders across Asia, and by the 18th century would refer to migrant Indian or Chinese laborers, and by the 19th century, would gain a new definition of the systematic shipping and hiring of Asian workers under contract on plantations that had been formerly worked by enslaved Africans. The word has had a variety of other implications and is sometimes regarded as offensive or a pejorative, depending upon the historical and geographical context; in India, its country of origin, it is still considered a derogatory slur. It is similar, in many respects, to the Spanish term peón, although both terms are used in some countries with different implications.

In [30]:
n_concordance_tokenised(text,'coolie') #native labourers, terms coined by the British

['prostitutes at all but from coolie women who hang about the ',
 'action being taken regarding them coolie women dr payne brings to ',
 'presence of filthy unregister ed coolie and beggar women detained there ',
 'durwan head sweeper sweeper mehtrani coolie total x xxviii statement showing ',
 'chaukidar clerk contractor compounder cook coolie cultivator constable day labourer domestic ',
 'thus mussamut bindee an agreement coolie was sent to the asylum ',
 'department for cart hire and coolie labour rs has been recovered ',
 'them of food if a coolie for instance who is given ',
 'secondly they are workmen doing coolie work thirdly they are vagabonds ',
 'abscess and heart diseasebodri kaneo coolie noncriminal male from darning admitted ',
 'patients males females total tea coolie cultivator sepoy trader beggar maid ',
 'a dwelling for the shop coolie who resides on the spot ',
 'to anybody phthisisghundu noncriminal garden coolie admitted on the april in ',
 'verified post mortem lu

In [46]:
n_concordance_tokenised(text,'keeper') #assisted doctors and surgeons in the asylums - both natives and europeans, mostly natives 

['midnapur goldsmith grocer mymensirgh i keeper asylum naga hills i jlilitary ',
 'fish seller goldsmith jail warder keeper asylum mason military overseas clerk ',
 'first man he met a keeper catching him on the stomach ',
 'has been obtained and each keeper has been able to look ',
 'hospital assistant compounder clerk head keeper male ditto female warders rs ',
 'and sold by the shop keeper ganja is also used for ',
 'requiring the presence of a keeper to guard the entrance during ',
 'r at rs duffadar jemadarni keeper at rs at rs at ',
 'surgeon compounder clerk head male keeper assistant head male keeper at ',
 'previous year an extra male keeper was entertained in the tezpur ',
 'sirear seller cycle reparing shop keeper blacksmith an basketmaker ter dar ',
 'intelligence and discipline among the keeper staff is still low at ',
 'male keeper assistant head male keeper at rs each ditto at ',
 'for discharge i am simply keeper and incidentally medical attendant and ',
 'subassistant 

In [47]:
n_concordance_tokenised(text,'superintendent')

['salary c of the lady superintendent placed at the disposal of ',
 'lahore j fairweather md surgeonmajor superintendent lunatic asylum no statement showing ',
 'i held the appointment of superintendent of vaccination punjab in october ',
 'my charge i was the superintendent of one central prison and ',
 'b scriven surgeonmajor the january superintendent lunatic asylum no return showing ',
 'a p rs a p superintendent deputy superintenden overseer matron writer ',
 'monthly wages rs a p superintendent overseer matron subassistant surgeon clerk ',
 'shixlong plunted bt hlhklno press superintendent asbami table of contents introductory ',
 'through the deputycommissioner by the superintendent i am surprised that no ',
 'view is shared by the superintendent of the patna asylum further ',
 'to the discretion of the superintendent for the recreation of native ',
 'was appointed to officiate as superintendent general during the absence of ',
 'no such case the deputy superintendent says he re

### Causes of Mental Illness

#### "Ganja Mania"

The British psychiatrists seemed to believe that ganja and charas caused mental illness 'ganja mania':


- ganja(hemp): Ganja is a Hindi name for hemp, derived from the word ganjha, from the Sanskrit gañjā, referring to a "powerful preparation from cannabis sativa".The term ganja, one of the oldest and most commonly used synonyms for marijuana in the English language, dates to before 1689
- charas : Charas is the name given to a hashish form of cannabis which is handmade in the Indian subcontinent and Jamaica. It is a cannabis concentrate made from the resin of the cannabis plant (Cannabis sativa or Cannabis indica).
- bhang(is an edible preparation of cannabis originating from the Indian Subcontinent,its traditionally distributed during the spring festival of Maha Shivaratri and Holi.)

There was a distinction between what they called physical vs moral causes of lunacy

'Physical' causes:
- small-pox 
- venereal disease 
- typhoid
- alcohol abuse /narcotics


Some weird ones/ 'moral causes':
- masturbation,
- syphilis 
- witchcraft

In [20]:
n_concordance_tokenised(text,'ganja mania')

['lall brahman etah charas and ganja mania motilal ironmonger far akabad heredity ',
 'superintendent atmaram hindu sowar nagpur ganja mania the statement shows that this ',
 'us in attributing it to ganja mania tulsi bania cause unknown no ',
 'sahib mu salman cultivator cuddapah ganja mania bad the papers show that ',
 'hindu occupa tion unknown madras ganja mania fair the papers show that ',
 'das hindu peon balasore no ganja mania this is a criminal lunatic ',
 'saheb mu salman cultivator anantapur ganja mania fair the papers show this ',
 'hindu cultivator bijapur smok ing ganja mania good this man was arrested ',
 'debauch in an attack of ganja mania slew seven of his nearest ',
 'singh rajput cultivator bhagalpur no ganja mania bad the medical certificate shows ',
 'gond cul tivator chhindwa ra ganja mania the memorandum shows that this ',
 'lalta hindu bhat servant lucknow ganja mania fair the statement shows that ',
 'one of more than probable ganja mania the chief points are 

In [5]:
# what are the words around 'asylum'
t = Text(corpus_tokens)
t.concordance('ganja', lines=50)  # by default NLTK's concordance method displays 25 lines

Displaying 50 of 17040 matches:
arcotic products of the hemp plant ( ganja , charas and bhang ), between the di
howing separatelyiv have no recorded ganja history confess to the use of the dr
al excessive consumers of charas and ganja , but not of bhang . Petty thefts an
owers 11 Hindu 42 Muhammadan 9 Smoke ganja . No one takes charas , which is not
 opinion against the moderate use of ganja among the sepoys . 39 . It is popula
y conceal the consumption of bhang , ganja , or charas ; and , as a rule , thos
and 2 Mahrattas ( combatants ) smoke ganja only in moderation mixed with tobacc
erience , on the decrease . 28 . The ganja - smokers consume about half a pice 
i , and other Hindu festivals . 33 . Ganja is smoked with tobacco in a dry stat
pers of Mahadev . 41 . In moderation ganja and bhang are con - sidered to be be
es not produce noxious effects , but ganja and charas - smoking produces irrita
 not get any of these diseases , but ganja and charas - smokers get bron - chit
loy . Th

In [32]:
n_concordance_tokenised(text,'ganja')

['be sold to the licensed ganja contractor at a fair valuation ',
 'in the concan yes only ganja and bhang are prepared the ',
 'of charas but not of ganja or siddhi these two may ',
 'the habitual excessive use of ganja may affect the children of ',
 'continued though moderate use of ganja appears to dry up the ',
 'of exces sive doses of ganja leading to homicidal frenzy this ',
 'found now they get round ganja for smoking flat ganja can ',
 'to the excessive use of ganja and bhang and in the ',
 'seeds found in the rajshahi ganja i have been assured by ',
 'per cent the effects of ganja and charas are nearly the ',
 'round ganja for smoking flat ganja can not be had round ',
 'excessive indulgence in majum and ganja smoking incite to unpremeditated crime ',
 'that the moderate use of ganja and charas is injurious it ',
 'to giving an opinion about ganja i can not say as ',
 'can not be had round ganja is im ported in preference ',
 'same charas is made from ganja tree and is somewha

In [35]:
n_concordance_tokenised(text,'witchcraft')

['influenced by superstition they assign witchcraft they are ignorant and have ',
 'they do not see except witchcraft they give heat as a ',
 'position to brstudy magic and witchcraft he was suffering from all ',
 'they do not see except witchcraft whose powers of observation are ']

In [36]:
n_concordance_tokenised(text,'superstition')

['tress there is a popular superstition that the use of ganja ',
 'admits thatpart i general report superstition of the natives as to ',
 'often they are influenced by superstition they assign witchcraft they are ',
 'their social relations habits and superstition they differ but slightly from ',
 'adverse and above all agelong superstition and prejudice formed a barrier ']

In [37]:
n_concordance_tokenised(text,'magic')

['the public works department a magic lantern and some native musical ',
 'were allowed as pets and magic lantern and grammophone entertainments were ',
 'and were periodically entertained with magic lantern exhibitions sweetmeats and fruits ',
 'addition a grama phone and magic lantern is relied upon for ',
 'theatricals nautches gramophone recitals and magic lantern entertainments were provided sweets ',
 'and with this object a magic lantern books native musical instruments ',
 'on the purchase of a magic lantern books musical instruments and ',
 'owns a gramophone and a magic lantern which are unfortunately both ',
 'gardening amusement in nagpur seven magic lantern entertainments were given during ',
 'performance c marionette entortainment d magic lantern entertainments with songs and ',
 'in a position to brstudy magic and witchcraft he was suffering ',
 'for the holding of concerts magic lantern exhibitions penny readings o ',
 'were also sent for examination magic performance 

### How does Lunancy exhibit?

In [33]:
n_concordance_tokenised(text,'melancholia')

['acute mania b chronic mania melancholia a acute dementia chbonic dementia ',
 'berar during the year function melancholia acute intermittent chronic associated statement ',
 'the year iidisorders i functions melancholia acute intermittent chronic stupor agi ',
 'idiocy mania acute or chronic melancholia acute or chronic dementia including ',
 'epileptics included dementia epileptics included melancholia is idiocy mental stupor and ',
 'acute mania b chronic mania melancholia a acute dementia b chronic ',
 'idiotcy mania acute or chronic melancholia acute or chronic dementia including ',
 'the year of function chronic melancholia acute recurrent or chronic circular ',
 'year acute mania chronic mania melancholia acute dementia chronic dementia idiotcy ',
 'the largest number next comes melancholia and then dementia vide statement ',
 'acute mania b chronic mania melancholia a acute dementia b chronic ',
 'acute mania acute dementia and melancholia the bulk of the population ',
 'the p

In [34]:
n_concordance_tokenised(text,'imbecility')

['pabalysis of the insane idioicy imbecility puerperal mania not insane total ',
 'preparations or derivatives delusional insanity imbecility dementia total predisposing causes were ',
 'under the divisions idiotcy and imbecility mania is the most fatal ',
 'melancholia dementia in cluding acquired imbecility idiocy epileptic insanity toxic insanity ',
 'a idiocy moral congo utal imbecility o puerperal m epileptic lt ',
 'mania melancholia dementia including acquired imbecility idiocy epileptic insanity toxic insanity ',
 'wilson pte foot ditto england imbecility unknown charles perrot pte hussars ',
 'of development ii ith idiocy imbecility mania acute intermittent chronic associated ',
 'of development ii disorde idiocy imbecility mania acute intermittent chronic associated ',
 'dementia idiocy mania c epilepsy imbecility amentia c epilepsy dementia c ',
 'paralysis of the insane idiotcy imbecility puerperal mania not insane total ',
 'chronic class s moral congenital imbecility a tu

In [39]:
n_concordance_tokenised(text,'masturbation')

['epilpsy congenital heat childbirth injury masturbation illness syphilis m f t ',
 'epilepsy paralysis congenital want illusage masturbation grief loss property losses in ',
 'ditary causes fever congenital disease masturbation destitution injury to head sunstroke ',
 'fever each to sunstroke and masturbation to injury to head each ',
 'intemperance hereditary epilepsy narcotic drugs masturbation childbirth other causes total moral ',
 'heredity epilepsy congenital heat traumatic masturbation phthisis m f t m ',
 'ditary causes fever congenital disease masturbation injury to head and destitution ',
 'of insanity among the lunatic masturbation debility taking excess of quinine ',
 'epilepsy congenital heat childbirth injury masturbation illness syphilis m f t ',
 'other intoxicants fever heredity epilepsy masturbation study m f t m ',
 'congenital heat poverty childbirth injury masturbation debility illness f t m ',
 'in fever in destitution in masturbation in syphilis in injury to ',


### 'Restraints' in British India Lunatic Asylums

In [25]:
n_concordance_tokenised(text,'moral treatment')

['on in the medical and moral treatment of the patients the want ',
 'notable a feature in the moral treatment of lunatics in europe and ']

In [26]:
n_concordance_tokenised(text,'lunatic labour') 

['in time and this by lunatic labour the limited accommodation brought to ',
 'the actual net profit of lunatic labour in the asylum during the ',
 'general for articles sold remitted lunatic labour and profit adjusted total grand ',
 'general for articles sold and lunatic labour total total poona received from ',
 'store and purchased value of lunatic labour newar rs rs balance rs ',
 'do sale of garden produce lunatic labour wood cutting c ldeduct cost ',
 'for articles sold or for lunatic labour value of manufactured goods made ',
 'abstract of estimated value of lunatic labour for nature of occupation amount ',
 'net receipt on account of lunatic labour supplied to the asylum contractor ',
 'abstract of estimated value of lunatic labour in delhi asylum for which ',
 'paddy and other grains c lunatic labour miscellaneous receipts gunny bags and ',
 'rs a p carpentering gardening lunatic labour extracting oil soorkee pounding basket ',
 'the public works department by lunatic labour 

In [40]:
n_concordance_tokenised(text,'mischief') 

['voluntary causing grievous hurt housebreaking mischief and assault miscellaneous offences males ',
 'voluntarily causing grievous hurt housebreaking mischief and assault miscellaneous offences males ',
 'who have done so much mischief are vide dr fairlands para ',
 'voluntarily causing grievous hurt housebreaking mischief and assault males females males ',
 'voluntarily causing grievous hurt housebreaking mischief and assault miscellaneous offences males ',
 'it is doing incalculably more mischief in these districts the informants ',
 'began to give trouble committed mischief abused people these things went ',
 'grievous hurt males housebreaking males mischief and assault and males miscellaneous ',
 'to housebreaking escape from jail mischief giving false information to the ',
 'males with housebreaking males with mischief and assault and males and ',
 'it keeps them out of mischief by placing them directly under ',
 'voluntarily causing grievous hurt housebreaking mischief and assau

In [41]:
n_concordance_tokenised(text,'excited') 

['the bhang appears to have excited him to madness he beat ',
 'that the man occasionally gets excited and dangerous is sub ject ',
 'a good man he becomes excited and loses temper as the ',
 'its fibre and that they excited themselves by inhaling its vapour ',
 'observation however he again became excited and noisy this alternating with ',
 'supervene the sexual passions are excited oftentimes leading to excessive debauchery ',
 'also developed leprosy the case excited much interest and many people ',
 'and stupid but at times excited had fever after admission quieted ',
 'feet a man who is excited with mental disease which has ',
 'with nitrogen gas inhalation in excited cases was so great that ',
 'on augusthe talks nonsense habitsdirty excited on julythe next entry is ',
 'change august lbs became very excited and abusive on the august ',
 'a person whose temper was excited by the use of ganja ',
 'said that he was frequently excited and that his excite ment ',
 'date barometer max

### Treatments of Lunacy

In [13]:
# what are the words around 'asylum'
t = Text(corpus_tokens)
t.concordance('convulsion', lines=38)  # by default NLTK's concordance method displays 25 lines 

Displaying 28 of 28 matches:
debility ( 2 ), bronchitis ( 1 ), convulsion ( 1 ), and cholera ( 1 ). Only a 
currence daily of an epileptiform convulsion which lasted for some minutes . M
 - 97 Rapid shallow respiration , convulsion . 6th 27 Female 25 26 - 4 - 97 Al
e 25 26 - 4 - 97 All food refused convulsion . 4th 28 Child 9 26 - 4 - 97 Tons
ld 9 26 - 4 - 97 Tonsils inflamed convulsion . 2nd 30 Female 14 30 - 4 - 97 2n
s but parts of a great functional convulsion for the ejectment of a poison who
ver . Heat Apoplexy . Dentition . Convulsion . Meningitis and Hydro - cephalus
ver . Heat Apoplexy . Dentition . Convulsion . Meningitis and Hydro - cephalus
ver . Heat Apoplexy . Dentition . Convulsion . Meningitis and Hydro - cephalus
ver . Heat Apoplexy . Dentition . Convulsion . Meningitis and Hydro - cephalus
ver . Heat Apoplexy . Dentition . Convulsion . Meningitis and Hydro - cephalus
 . Heat Apoplexy . De . ntition . Convulsion . Meningitis and Hydro - eephalus
ed Fevers . Apoplexy . 

In [42]:
n_concordance_tokenised(text,'Electric Convulsion Therapy') 

['to december with phrenozal and electric convulsion therapy disease total treated recover ed ']

In [44]:
n_concordance_tokenised(text,'Cardiazal') 

[]

In [ ]:
n_concordance_tokenised(text,' ') 